<a href="https://colab.research.google.com/github/vikashnitk/Fake-News-Classifier-LSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('/content/drive/MyDrive/ML_projects/FakeNewsData/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
###Drop Nan Values
df=df.dropna()

In [8]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.5.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [16]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

### Embedding Representation

In [34]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
#print(embedded_docs)

In [35]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2992,
       1219, 1494, 3015,  817,  672, 2321, 1395, 3404, 1642], dtype=int32)

In [36]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
len(embedded_docs),y.shape

(18285, (18285,))

In [38]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [39]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [41]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 11ms/step - loss: 0.3409 - accuracy: 0.8373 - val_loss: 0.1956 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1525 - accuracy: 0.9429 - val_loss: 0.1883 - val_accuracy: 0.9236
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1118 - accuracy: 0.9586 - val_loss: 0.2105 - val_accuracy: 0.9220
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0875 - accuracy: 0.9667 - val_loss: 0.2223 - val_accuracy: 0.9201
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0693 - accuracy: 0.9760 - val_loss: 0.2423 - val_accuracy: 0.9183
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0536 - accuracy: 0.9820 - val_loss: 0.2924 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0433 - accuracy: 0.9853 - val_loss: 0.3318 - val_accuracy: 0.9140
Epoch

### Adding Dropout 

### Performance Metrics And Accuracy

In [42]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
confusion_matrix(y_test,y_pred)

array([[3112,  307],
       [ 237, 2379]])

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9098591549295775